 # Setup

In [1]:
import warnings
# warnings.filterwarnings("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=RuntimeWarning)

import sqlite3
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt


path = "../input/soccer/"
database = path + 'database.sqlite'
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT * FROM sqlite_master
                        WHERE type='table';""", conn)

pd.set_option('display.float_format', '{:.4f}'.format)

In [2]:
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [3]:
# Fetching required data tables, leagues together with countries

leagues = pd.read_sql("""SELECT l.id, l.country_id, c.name as country_name, l.name as league_name
                         FROM League l JOIN Country c ON c.id = l.country_id;""", conn, index_col='id')
team_df = pd.read_sql("""SELECT id, team_api_id, team_long_name, team_short_name 
                        FROM Team;""", conn, index_col='id')

player_df = pd.read_sql("SELECT * FROM Player;", conn, index_col='id')

In [4]:
### Fetching 'Player_Attributes'
# - keeping only rows with 'overall_rating'
# - adding second date, as a 'valid to' date, the date when the attribure changed

player_attr_df = pd.read_sql(
                    """SELECT id, player_api_id, player_fifa_api_id, date(date) as 'date', 
                    	   coalesce(date(lead(date) over(partition by player_api_id order by date)), date('now')) as 'valid_to',
                    	   overall_rating, potential, preferred_foot, attacking_work_rate,
                           defensive_work_rate, crossing, finishing, heading_accuracy,
                           short_passing, volleys, dribbling, curve, free_kick_accuracy,
                           long_passing, ball_control, acceleration, sprint_speed,
                           agility, reactions, balance, shot_power, jumping, stamina,
                           strength, long_shots, aggression, interceptions, positioning,
                           vision, penalties, marking, standing_tackle, sliding_tackle,
                           gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes
                    FROM  Player_Attributes
                    WHERE overall_rating IS NOT NULL;""", conn, index_col='id')
player_attr_df['date'] = pd.to_datetime(player_attr_df['date'])
player_attr_df['valid_to'] = pd.to_datetime(player_attr_df['valid_to'])

In [5]:
### Fetching 'Match'
match_df = pd.read_sql("SELECT * FROM Match;", conn)
match_df['date'] = pd.to_datetime(match_df['date'])
match_df['result'] = np.sign(match_df['home_team_goal'] - match_df['away_team_goal']) \
                    .map({1: 'home', -1: 'away', 0: 'draw'})
match_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Data columns (total 116 columns):
 #    Column            Dtype         
---   ------            -----         
 0    id                int64         
 1    country_id        int64         
 2    league_id         int64         
 3    season            object        
 4    stage             int64         
 5    date              datetime64[ns]
 6    match_api_id      int64         
 7    home_team_api_id  int64         
 8    away_team_api_id  int64         
 9    home_team_goal    int64         
 10   away_team_goal    int64         
 11   home_player_X1    float64       
 12   home_player_X2    float64       
 13   home_player_X3    float64       
 14   home_player_X4    float64       
 15   home_player_X5    float64       
 16   home_player_X6    float64       
 17   home_player_X7    float64       
 18   home_player_X8    float64       
 19   home_player_X9    float64       
 20   home_player_X10   float64 

In [6]:
# Identifying columns for further work

betting_cols = match_df.columns[85:].tolist()
players_cols = match_df.columns[11:77].tolist()
add_info_cols = match_df.columns[77:85].tolist()

cols_to_drop = betting_cols + add_info_cols

# Players Analysis

In [7]:
# Cleaning match_df for 'Players Analysis'
match_players = match_df.drop(columns=cols_to_drop)
match_players = match_players[match_players['home_player_X1'].notna()]

In [8]:
# Identify all relevant coordinate columns
coord_cols = [
    col for col in match_players.columns
    if col.startswith("home_player_X") or
       col.startswith("home_player_Y") or
       col.startswith("away_player_X") or
       col.startswith("away_player_Y")
]

id_columns = [f'home_player_{i}' for i in range(1, 12)] + \
             [f'away_player_{i}' for i in range(1, 12)]

# Fill NaN with 0 and convert to int
match_players[coord_cols] = match_players[coord_cols].fillna(0).astype(int)
match_players[id_columns] = match_players[id_columns].fillna(0).astype('int32') 

# Create tuple columns for players
for i in range(1, 12):
    match_players[f'home_player_{i}_pos'] = list(zip(match_players[f'home_player_X{i}'], 
                                                     match_players[f'home_player_Y{i}']))

for i in range(1, 12):
    match_players[f'away_player_{i}_pos'] = list(zip(match_players[f'away_player_X{i}'], 
                                                     match_players[f'away_player_Y{i}']))

Home_player_id X1-X11 - The ID of the player on each position which can be linked to the FIFA stats

Away_player_id X1-X11 - 30 Player-specific FIFA statistics

Home_player_X1 – X11
For each home player the position in the width of the Field (1-9)

Away_player_X1 – X11
For each away player the position in the width of the Field (1-9)

Home_player_Y1 – Y11
For each home player the position in the length of the Field (1-11)

Away_player_Y1 – Y11
For each away player the position in the length of the Field (1-11)

![](https://i125.fastpic.org/big/2025/0808/e0/091e1e44d61c2c05084c817769b3a1e0.png)

In [9]:
# Detect goalkeeper IDs
def find_goalkeeper_id(row, side):
    for i in range(1, 12):
        if row[f'{side}_player_{i}_pos'] == (1, 1):
            return row[f'{side}_player_{i}']  # ID from the ID column
    return None

match_players['home_gk_id'] = match_players.apply(find_goalkeeper_id, 
                                                  side='home', axis=1).astype('Int32')
match_players['away_gk_id'] = match_players.apply(find_goalkeeper_id, 
                                                  side='away', axis=1).astype('Int32')


In [10]:
unique_gk_ids = pd.unique(pd.concat([match_players['home_gk_id'], 
                                     match_players['away_gk_id']]).dropna())
unique_gk_ids = unique_gk_ids[unique_gk_ids != 0] 

goalkeepers = pd.DataFrame({
    'player_id': unique_gk_ids.astype('int32'),
    'position': 'goalkeeper'
}).reset_index(drop=True)

In [11]:
# Merge goalkeepers info into player_attr_df
player_attr_df = player_attr_df.merge(
    goalkeepers[['player_id', 'position']],
    left_on='player_api_id',
    right_on='player_id',
    how='left'
)

player_attr_df.drop(columns='player_id', inplace=True)

player_attr_df['position'] = player_attr_df['position'].fillna('other')

In [12]:
player_attr_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183142 entries, 0 to 183141
Data columns (total 43 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   player_api_id        183142 non-null  int64         
 1   player_fifa_api_id   183142 non-null  int64         
 2   date                 183142 non-null  datetime64[ns]
 3   valid_to             183142 non-null  datetime64[ns]
 4   overall_rating       183142 non-null  int64         
 5   potential            183142 non-null  int64         
 6   preferred_foot       183142 non-null  object        
 7   attacking_work_rate  180748 non-null  object        
 8   defensive_work_rate  183142 non-null  object        
 9   crossing             183142 non-null  int64         
 10  finishing            183142 non-null  int64         
 11  heading_accuracy     183142 non-null  int64         
 12  short_passing        183142 non-null  int64         
 13  volleys       

In [13]:
player_attr_df['preferred_foot'].value_counts(dropna=False)

preferred_foot
right    138409
left      44733
Name: count, dtype: int64

In [14]:
player_attr_df['attacking_work_rate'].value_counts(dropna=False)

attacking_work_rate
medium    125070
high       42823
low         8569
None        3639
None        2394
norm         348
y            106
le           104
stoc          89
Name: count, dtype: int64

In [15]:
player_attr_df['defensive_work_rate'].value_counts(dropna=False)

defensive_work_rate
medium    130846
high       27041
low        18432
_0          2394
o           1550
1            441
ormal        348
2            342
3            258
5            234
7            217
0            197
6            197
9            152
4            116
es           106
ean          104
tocky         89
8             78
Name: count, dtype: int64

# Betting Analysis

In [16]:

# betting_df = pd.read_sql("""SELECT
# id,country_id,league_id,season,stage,date,match_api_id,
# home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,
# B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,
# WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
# FROM MATCH""", conn)

# To Be Continued ...